In [12]:
import os
from pandas import read_csv, read_excel, DataFrame
import sae
import gurobipy as gp
from gurobipy import GRB
import numpy as np

import sys
sys.path.append('../../')
import saedfsc

In [13]:
numCustomersTypes = 25
numObjectives = 10

# Create a list of supplier names
names = ['CustomerType' + str(i) for i in range(1, numCustomersTypes + 1)]

qty = np.random.randint(low=25, high=1000, size=numCustomersTypes)
priceFocus = np.random.random(size=numCustomersTypes)

wts = [np.array2string(np.random.random(numObjectives), separator=',') for _ in range(numCustomersTypes)]

# Create the DataFrame
df = DataFrame({
    'Name': names,
    'Quantity': qty,
    'PriceFocus': priceFocus,
    'PerformanceUtilityWeights': wts
})
df.to_csv('../resources/customers.csv', index=False)
customers: DataFrame = read_csv('../resources/customers.csv')
customers

,Name,Quantity,PriceFocus,PerformanceUtilityWeights
0,CustomerType1,207,0.977040,"[0.52987363,0.35178429,0.70986886,0.02267754,0..."
1,CustomerType2,362,0.582443,"[0.81197919,0.32330553,0.67095492,0.91466114,0..."
2,CustomerType3,56,0.205295,"[0.93328301,0.80693842,0.18938607,0.58569296,0..."
3,CustomerType4,282,0.907159,"[0.32733468,0.16841703,0.82616086,0.70444859,0..."
4,CustomerType5,622,0.747278,"[0.08127728,0.41932527,0.76296338,0.23393942,0..."
5,CustomerType6,363,0.878238,"[0.75942505,0.42101603,0.91910032,0.96802809,0..."
6,CustomerType7,985,0.457650,"[0.68510661,0.46255854,0.73891313,0.71986678,0..."
7,CustomerType8,766,0.752265,"[0.31847945,0.89463808,0.28893099,0.58307066,0..."
8,CustomerType9,116,0.364113,"[0.57931748,0.8016078 ,0.33964168,0.4015043 ,0..."
9,CustomerType10,763,0.501717,"[0.99861929,0.3825658 ,0.04026279,0.8431939 ,0..."


Print dictionary of customers and their weights.

In [15]:
name_weights_dict = customers.set_index('Name')['PerformanceUtilityWeights'].to_dict()
cWts = {c : np.fromstring(name_weights_dict[c].strip('[]'), sep=',') for c in name_weights_dict}
cWts

{'CustomerType1': array([0.52987363, 0.35178429, 0.70986886, 0.02267754, 0.60650101,
        0.43933436, 0.62306168, 0.89958702, 0.7719935 , 0.46308688]),
 'CustomerType2': array([0.81197919, 0.32330553, 0.67095492, 0.91466114, 0.74437359,
        0.75613038, 0.91323541, 0.46551214, 0.76357209, 0.55667932]),
 'CustomerType3': array([0.93328301, 0.80693842, 0.18938607, 0.58569296, 0.50806153,
        0.44122945, 0.79523984, 0.62624826, 0.89824096, 0.14083512]),
 'CustomerType4': array([0.32733468, 0.16841703, 0.82616086, 0.70444859, 0.80802148,
        0.82961468, 0.60212316, 0.84200579, 0.91175249, 0.86761008]),
 'CustomerType5': array([0.08127728, 0.41932527, 0.76296338, 0.23393942, 0.67012482,
        0.01352868, 0.03791778, 0.89535719, 0.9762528 , 0.13023384]),
 'CustomerType6': array([0.75942505, 0.42101603, 0.91910032, 0.96802809, 0.71116034,
        0.65260111, 0.66898952, 0.4594247 , 0.37547805, 0.15927617]),
 'CustomerType7': array([0.68510661, 0.46255854, 0.73891313, 0.7198667

In [17]:
cQty = dict(zip(customers['Name'], customers['Quantity'])) # customer quantities
cQty

{'CustomerType1': 207,
 'CustomerType2': 362,
 'CustomerType3': 56,
 'CustomerType4': 282,
 'CustomerType5': 622,
 'CustomerType6': 363,
 'CustomerType7': 985,
 'CustomerType8': 766,
 'CustomerType9': 116,
 'CustomerType10': 763,
 'CustomerType11': 694,
 'CustomerType12': 505,
 'CustomerType13': 656,
 'CustomerType14': 451,
 'CustomerType15': 542,
 'CustomerType16': 735,
 'CustomerType17': 392,
 'CustomerType18': 91,
 'CustomerType19': 69,
 'CustomerType20': 419,
 'CustomerType21': 531,
 'CustomerType22': 513,
 'CustomerType23': 259,
 'CustomerType24': 523,
 'CustomerType25': 182}